In [54]:
from Executor import *


executor = ExecutorBuilder().\
    with_runID("first").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()

executor.load_model_from_file() # loads a finetuned model that has been trained for 5 epochs
 
executor.tune_softmax_layer_for_epochs(1) # just another run to ensure everything's right

executor.load_precomputed_conv_models()

Pretrained Vgg16 model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3
Epoch 1/1
6255/6255 [==============================] - 187s - loss: 1.0607 - acc: 0.5361 - val_loss: 0.9428 - val_acc: 0.5190
Vgg model finetuned.
loading precomputed conv. outputs...
done...


<Executor.Executor instance at 0x7f26107e1200>

In [ ]:
def proc_wgts(layer): return [o for o in layer.get_weights()]

def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=executor.conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
        ])

    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))
    return model

In [59]:

### Experiments with precomputed conv model outputs.
## THE OBJECTIVE IS TO BE ABLE TO SAVE/LOAD PRECOMPUTED OUTPUT FROM CONV. LAYERS.
## RIGHT NOW GIVING EXTREMELY LARGE LOG-LOSS

linearModel = get_fc_model()
linearModel.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])

j =0
for layer in linearModel.layers:
    j+=1
    if(j==7):        
        print("setting last layer being trainable")
        layer.trainable=True
    else:
        layer.trainable=False
        

for layer in linearModel.layers:
    if layer.trainable:
        print("found trainable layer: ",layer)

linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

# #---------- doesn't work.. wind up getting large log-loss ---------------------

setting last layer being trainable
('found trainable layer: ', <keras.layers.core.Dense object at 0x7f260b279f10>)
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 8s - loss: 1.1126 - acc: 0.4686 - val_loss: 1.0323 - val_acc: 0.5113
Epoch 2/5
6255/6255 [==============================] - 8s - loss: 1.0211 - acc: 0.5125 - val_loss: 1.0208 - val_acc: 0.5094
Epoch 3/5
6255/6255 [==============================] - 8s - loss: 0.9981 - acc: 0.5196 - val_loss: 1.0390 - val_acc: 0.5010
Epoch 4/5
6255/6255 [==============================] - 8s - loss: 0.9624 - acc: 0.5305 - val_loss: 1.0698 - val_acc: 0.5003
Epoch 5/5
6255/6255 [==============================] - 8s - loss: 0.9072 - acc: 0.5701 - val_loss: 1.0585 - val_acc: 0.5061


In [60]:
linearModel.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 10, validation_data=(executor.val_precomputed, executor.val_labels))


Train on 6255 samples, validate on 1549 samples
Epoch 1/10
6255/6255 [==============================] - 3s - loss: 0.8255 - acc: 0.6006 - val_loss: 1.0558 - val_acc: 0.5048
Epoch 2/10
6255/6255 [==============================] - 3s - loss: 0.8156 - acc: 0.6106 - val_loss: 1.0554 - val_acc: 0.5016
Epoch 3/10
6255/6255 [==============================] - 3s - loss: 0.8163 - acc: 0.6082 - val_loss: 1.0554 - val_acc: 0.4990
Epoch 4/10
6255/6255 [==============================] - 3s - loss: 0.8092 - acc: 0.6225 - val_loss: 1.0557 - val_acc: 0.4964
Epoch 5/10
6255/6255 [==============================] - 3s - loss: 0.8034 - acc: 0.6197 - val_loss: 1.0564 - val_acc: 0.4906
Epoch 6/10
6255/6255 [==============================] - 3s - loss: 0.8075 - acc: 0.6216 - val_loss: 1.0574 - val_acc: 0.4893
Epoch 7/10
6255/6255 [==============================] - 3s - loss: 0.8050 - acc: 0.6232 - val_loss: 1.0582 - val_acc: 0.4861
Epoch 8/10
6255/6255 [==============================] - 3s - loss: 0.7968 - a